In [54]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [55]:
df = pd.read_csv(r"D:\vs code\Projects\MachineLearning\Spamclassifier\spam.csv",encoding="ansi")

In [56]:
df.head()
df = df.drop(columns=["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"])

In [57]:
df['v1'].value_counts()

v1
ham     4825
spam     747
Name: count, dtype: int64

Lowercasing

In [58]:
columns_to_lowercase = ['v1', 'v2']
for col in columns_to_lowercase:
    if col in df.columns:
        if pd.api.types.is_string_dtype(df[col]):
            df[col] = df[col].astype(str).str.lower()
            print(f"Column '{col}' has been lowercased.")
df.head()

Column 'v1' has been lowercased.
Column 'v2' has been lowercased.


,v1,v2
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."


Url removal

In [65]:

import re
url_pattern = re.compile(r'https?://\S+|www\.\S+|\S+\.(com|org|net|gov|edu|uk|ca|in|io|co|biz|info|xyz|me|app|dev|blog|shop|online|site|store|tech|club|live|solutions|ai|io|cloud)\b')
def remove_urls(text):
    text = str(text) # Ensure text is string
    return url_pattern.sub(r'', text)
df['v2_nourl'] = df['v2'].apply(remove_urls)
df['v2_nourl'][15]

'xxxmobilemovieclub: to use your credit, click the wap link in the next txt message or click here>>  ?n=qjkgighjjgcbl'

Punctuation removal

In [ ]:
def clean_text(series):
    return series.str.translate(str.maketrans('', '', string.punctuation)).str.strip()

df['v2_clean'] = df['v2_nourl'].apply(clean_text)
df['v2_clean'].head()

0    go until jurong point crazy available only in ...
1                              ok lar joking wif u oni
2    free entry in 2 a wkly comp to win fa cup fina...
3          u dun say so early hor u c already then say
4    nah i dont think he goes to usf he lives aroun...
Name: v2_clean, dtype: object

Chat Word Removal

In [68]:
chat_words_map = {
    'lol': 'laughing out loud',
    'brb': 'be right back',
    'g2g': 'got to go',
    'btw': 'by the way',
    'omg': 'oh my god',
    'wtf': 'what the fuck',
    'u': 'you',
    'r': 'are',
    'ur': 'your',
    'lyk': 'like',
    '2': 'to',
    '4': 'for',
    'cya': 'see you',
    'msg': 'message',
    'pls': 'please',
    'plz': 'please',
    'thx': 'thanks',
    'ty': 'thank you',
    'coz': 'because',
    'b4': 'before',
    'idk': 'i do not know',
    'imo': 'in my opinion',
    'irl': 'in real life',
    'fyi': 'for your information',
    'np': 'no problem',
    'ttyl': 'talk to you later',
    'wbu': 'what about you',
    'c' : 'can',
    'n':'and',
    'k':'ok',
    'dun' : 'dont'
}
def expand_word(text):
    words = text.split()
    expanded_words = []
    for word in words:
        if word in chat_words_map.keys():
            expanded_words.append(chat_words_map[word])
        else:
            expanded_words.append(word)
    return ' '.join(expanded_words)

df['v2_expChat'] = df['v2_clean'].apply(expand_word)
df['v2_expChat'].head()

0    go until jurong point crazy available only in ...
1                            ok lar joking wif you oni
2    free entry in to a wkly comp to win fa cup fin...
3    you dont say so early hor you can already then...
4    nah i dont think he goes to usf he lives aroun...
Name: v2_expChat, dtype: object

Removal Of Stop Word

In [79]:
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = list(stopwords.words('english'))
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)
df['v2_nostop'] = df['v2_expChat'].apply(remove_stopwords)
df['v2_nostop'].head()

0    go jurong point crazy available bugis great wo...
1                                ok lar joking wif oni
2    free entry wkly comp win fa cup final tkts 21s...
3                       dont say early hor already say
4          nah dont think goes usf lives around though
Name: v2_nostop, dtype: object

Steming